# Request Financial Statement Datasets

In [3]:
import bql
import pandas as pd
import utils.data_request_helper as helper
from utils.s3_helper import S3Helper

import os
import importlib
import json

from ipywidgets import IntProgress
from IPython.display import display

In [48]:
importlib.reload(helper)

<module 'utils.data_request_helper' from '/project/utils/data_request_helper.py'>

### Quarterly Data for the Dow Jones

In [49]:
# Index to use for point in time firms
index = 'INDU Index'
filename = 'data_quarterly.json'
reporting_period = 'Q'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]

In [50]:
data_helper = helper.FinancialDataRequester(index_id=index,
                                            dataset_name='quarterly_pit_indu_blended',
                                            rebalance_dates=rebalance_dates,
                                            reporting_frequency=reporting_period)

In [51]:
df_rebalance_dates = data_helper.get_rebalance_dates()

100%|██████████| 24/24 [00:14<00:00,  1.64it/s]

In [52]:
all_data = data_helper.create_financial_dataset()

100%|█████████▉| 424/425 [25:51<00:03,  3.60s/it]

#### Save to Bloomberg Lab S3 Storage

In [54]:
# Write the data to local ephemeral storage
local_file = '/tmp/dow_quarterly_ltm_v2.json'
with open(local_file, 'w') as f:
    json.dump(all_data, f)

In [55]:
# Create S3 Helper object
s3_helper = S3Helper('tmp/fs')

In [56]:
# Upload to Bloomberg Lab S3 Storage
s3_helper.add_file(local_filename=local_file)

### Annual Data for the Dow Jones

In [38]:
# Index to use for point in time firms
index = 'INDU Index'
filename = 'data_annual_pit_dow.json'
reporting_period = 'A'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]

In [41]:
data_helper = helper.FinancialDataRequester(index_id=index,
                                            dataset_name='annual_pit_indu_blended',
                                            rebalance_dates=rebalance_dates,
                                            reporting_frequency=reporting_period)

In [43]:
all_data = data_helper.create_financial_dataset()

  4%|▍         | 1/24 [00:00<00:15,  1.51it/s]

DataRequest: Complete for 2024-12-31


  8%|▊         | 2/24 [00:01<00:19,  1.15it/s]

DataRequest: Complete for 2024-09-30


 12%|█▎        | 3/24 [00:02<00:16,  1.25it/s]

...

 96%|█████████▌| 23/24 [00:14<00:00,  1.72it/s]

DataRequest: Complete for 2019-06-30


100%|██████████| 24/24 [00:15<00:00,  1.50it/s]

DataRequest: Complete for 2019-03-31


 99%|█████████▉| 145/146 [08:46<00:03,  3.58s/it]

In [14]:
rebalance_dates_requested = data_helper.get_rebalance_dates()

100%|██████████| 2/2 [00:01<00:00,  1.61it/s]

In [44]:
all_data.keys()

dict_keys(['2020-07-22', '2020-07-24', '2020-07-31', '2020-08-06', '2020-08-12', '2020-10-15', '2020-10-28', '2020-10-29', '2020-10-30', '2020-11-12', '2021-01-20', '2021-01-21', '2021-01-26', '2021-01-27', '2021-01-29', '2021-02-03', '2021-02-09', '2021-02-11', '2021-02-12', '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23', '2021-02-25', '2021-02-26', '2021-03-24', '2021-06-24', '2021-07-27', '2021-07-30', '2021-08-04', '2021-08-18', '2021-09-09', '2021-10-14', '2021-10-26', '2021-10-28', '2021-11-10', '2022-01-14', '2022-01-18', '2022-01-20', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-02-03', '2022-02-04', '2022-02-08', '2022-02-09', '2022-02-12', '2022-02-15', '2022-02-16', '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25', '2022-03-01', '2022-03-11', '2022-03-18', '2022-06-27', '2022-07-26', '2022-07-29', '2022-08-17', '2022-09-08', '2022-10-13', '2022-10-28', '2022-11-16', '2022-11-29', '2023-01-13', '2023-01-17', '2023-01-24', '2023-01-

In [53]:
all_data['2020-07-22'].keys()

KeyError: '2020-07-22'

In [28]:
rebalance_dates_requested.loc[['2025-02-26', '2025-02-25']]

PERIOD_END_DATE
AS_OF_DATE ID                            
2025-02-26 GS UN Equity        2024-12-31
           NVDA UQ Equity      2025-01-26
2025-02-25 MRK UN Equity       2024-12-31